In [1]:
from qutip import *
import numpy as np
import matplotlib.pyplot as plt
from random import seed
from random import gauss
from itertools import product
seed(1)

In [2]:
#operator definitionis
N = 2
si = qeye(2)
sx = 0.5*sigmax()
sy = 0.5*sigmay()
sz = 0.5*sigmaz()

In [3]:
#make list of tensor products
sx_list = []
sy_list = []
sz_list = []

for n in range(N):
    op_list = []
    for m in range(N):
        op_list.append(si)

    op_list[n] = sx
    sx_list.append(tensor(op_list))

    op_list[n] = sy
    sy_list.append(tensor(op_list))

    op_list[n] = sz
    sz_list.append(tensor(op_list))


In [4]:
#Construct Hamiltonian
B = 0
A0 = 1
H = B*sz_list[0] 
for n in range(N-1):
    H += A0*sz_list[0]*sz_list[n+1] + A0*sx_list[0]*sx_list[n+1] + A0*sy_list[0]*sy_list[n+1]
print(H)

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.25  0.    0.    0.  ]
 [ 0.   -0.25  0.5   0.  ]
 [ 0.    0.5  -0.25  0.  ]
 [ 0.    0.    0.    0.25]]


In [5]:
#Find Groundstate
H.groundstate()

(-0.7500000000000001,
 Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
 Qobj data =
 [[ 0.        ]
  [ 0.70710678]
  [-0.70710678]
  [ 0.        ]])

In [11]:
#Make basis
Sbasis = []
for i in range(N):
    for j in range(N):
        Sbasis.append(tensor([basis(N,i),basis(N,j)]))
print(Sbasis)


[Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[1.]
 [0.]
 [0.]
 [0.]], Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.]
 [1.]
 [0.]
 [0.]], Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.]
 [0.]
 [1.]
 [0.]], Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[0.]
 [0.]
 [0.]
 [1.]]]


In [47]:
#get sz values for basis states
sz = np.zeros((2**N,N), dtype = complex)
a = [[1 for j in range(N)] for i in range(2**N)]
for i in range(2**N):
    for j in range(N):
        #matrix element <bra|Sz|ket>
        sz[i][j] = sz_list[j].matrix_element(Sbasis[i],Sbasis[i])

print(sz)

[[ 0.5+0.j  0.5+0.j]
 [ 0.5+0.j -0.5+0.j]
 [-0.5+0.j  0.5+0.j]
 [-0.5+0.j -0.5+0.j]]


In [49]:
#Define RBM Parameters
M = 1
alp = M/N

a = np.zeros(N, dtype = complex)
b = np.zeros(M, dtype = complex)
W = np.zeros((N,M), dtype = complex)


In [282]:
#Function to give RBM wavefuntion
def RBM_ansatz(a, b, W, Sbasis, spin, N, M):
    expTerm = np.zeros(2**N)
    visBias = np.zeros(2**N)
    WSum = np.zeros((2**N,M))
    hidSum = np.zeros((2**N,M))
    hidProduct = np.zeros(2**N)

    for i in range(2**N):
        for j in range (N):
            visBias[i] += spin[i][j]*a[j]
            for m in range(M):
                WSum[i][m] += W[m][j]*spin[i][j]
    for i in range(2**N):
        for m in range(M):
            hidSum[i][m] = np.cosh(WSum[i][m] + b[m])*2
            hidProduct[i] = np.prod(hidSum[i])
    for i in range(2**N):
        expTerm[i] = np.exp(visBias[i])


    #Wavefunction Definition
    psiMValues = np.zeros(2**N)
    for i in range(2**N):
        psiMValues[i] = expTerm[i]*hidProduct[i]
    psiM = Qobj(psiMValues)
    return psiM

psiM = RBM_ansatz(a, b, W, Sbasis, spin, N, M)
print(psiM)
 

Quantum object: dims = [[4], [1]], shape = (4, 1), type = ket
Qobj data =
[[148.78103854]
 [  3.08616127]
 [  3.08616127]
 [  0.41766651]]


In [283]:
#Variation Energy Definition
def varEnergy(psiM, H):
    E = psiM
    return E
E = varEnergy(psiM, H)
print (E)

print(H)
H.istp

#something happening with dimensions of matricies 



Quantum object: dims = [[4], [1]], shape = (4, 1), type = ket
Qobj data =
[[148.78103854]
 [  3.08616127]
 [  3.08616127]
 [  0.41766651]]
Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.25  0.    0.    0.  ]
 [ 0.   -0.25  0.5   0.  ]
 [ 0.    0.5  -0.25  0.  ]
 [ 0.    0.    0.    0.25]]


False

In [284]:
#Gradient Descent 
def updateParameters(a, b, W, learning rate):
    
    
    return a, b, W


SyntaxError: invalid syntax (<ipython-input-284-06f2cd0b43a3>, line 2)

In [ ]:
#Ground State Wavefunction
RBM_ansatz(a, b, W, Sbasis, spin, N, M)